In [3]:
import MDAnalysis as mda
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA
from MDAnalysis.analysis import hydrogenbonds
import sys
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from rich import print as rprint
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from MDAnalysis.analysis.rdf import InterRDF
from MDAnalysis.analysis import msd
import json
from rich.progress import track

from scipy.stats import gaussian_kde
from copy import deepcopy

# 读取轨迹

In [2]:
config_s = {
       "SPCE_qspcfw": {
          "fp_ncdf": "/home/iaw/DATA/ALP/bulk2/nvt-0.5fs/md20ps.ncdf"
        , "fp_top": "/home/iaw/DATA/ALP/bulk2/nvt-0.5fs/model.prmtop"
        , "start_frame": 0
        , "end_frame": 400000
        , "step": 100
    }
    ,  "SPCE_spcfw": {
          "fp_ncdf": "/home/iaw/DATA2/wat_md/bulk_spce_flex_spcfw/nvt_200ps/md200ps.ncdf"
        , "fp_top": "/home/iaw/DATA2/wat_md/bulk_spce_flex_spcfw/nvt_200ps/model.prmtop"
        , "start_frame": 0
        , "end_frame": 400000
        , "step": 100
    }
    ,  "OPC3POL": {
          "fp_ncdf": "/home/iaw/DATA2/wat_md/bulk_opc3pol_rigid/nvt_200ps/md200ps.ncdf"
        , "fp_top": "/home/iaw/DATA2/wat_md/bulk_opc3pol_rigid/nvt_200ps/model.prmtop"
        , "start_frame": 0
        , "end_frame": 400000
        , "step": 100
    }
    ,  "TIP3P": {
          "fp_ncdf":"/home/iaw/DATA2/wat_md/bulk_tip3p_rigid/216/5_nvt_200ps/md200ps.ncdf"
        , "fp_top": "/home/iaw/DATA2/wat_md/bulk_tip3p_rigid/216/5_nvt_200ps/model.prmtop"
        , "start_frame": 0
        , "end_frame": 400000
        , "step": 100
    }
    ,  "SPCE": {
          "fp_ncdf":"/home/iaw/DATA2/wat_md/bulk_spce_rigid/nvt_200ps/md200ps.ncdf"
        , "fp_top": "/home/iaw/DATA2/wat_md/bulk_spced_rigid/nvt_200ps/model.prmtop"
        , "start_frame": 0
        , "end_frame": 400000
        , "step": 100
    }
}

In [ ]:
result_s = deepcopy(config_s)
for k in track(config_s.keys()):
    i_trial = config_s[k]
    fp_ncdf = i_trial["fp_ncdf"]
    fp_top = i_trial["fp_top"]

    i_u = u = mda.Universe(fp_top
                           ,fp_ncdf
                           , dt = 0.0005, format= "NCDF"
                           , topology_format="PRMTOP")

    bulk_wat_O_sele = "type OW"
    bulk_O_wat = i_u.select_atoms(bulk_wat_O_sele, updating=False)
    rdf_O = InterRDF(g1=bulk_O_wat, g2=bulk_O_wat, nbins=100, range=[0,15], norm="rdf",exclude_same="residue")
    rdf_O.run(step=10)
    result_s[k]["result_OO"] = rdf_O

    bulk_wat_H_sele = "type HW"
    bulk_H_wat = i_u.select_atoms(bulk_wat_H_sele, updating=False)
    rdf_H = InterRDF(g1=bulk_H_wat, g2=bulk_H_wat, nbins=100, range=[0,15], norm="rdf",exclude_same="residue")
    rdf_H.run(step=10)
    result_s[k]["result_HH"] = rdf_H

Output()

## gr
- O and O
- H and H

In [ ]:
fig = plt.figure(figsize=(9,7), dpi=300)

plt.rcParams["axes.labelweight"] ="bold" 
plt.rcParams["font.family"]="Times New Roman" 
plt.rcParams["font.weight"]="bold" 
plt.rcParams["font.size"]=20
plt.rcParams["mathtext.default"]="regular"


ax1= plt.gca()

#ax1.spines['bottom'].set_position(('data',0))
ax1.spines['left'].set_position(('axes',0))
ax1.spines['bottom'].set_linewidth(2)
ax1.spines['left'].set_linewidth(2)
ax1.spines['right'].set_linewidth(2)
ax1.spines['top'].set_linewidth(2)

ax1.tick_params(which='major'          # 设置对主刻度和次刻度生效
                ,bottom=True           # 底部坐标轴
                ,top=True
                ,left=True             # 左侧坐标轴
                ,right=True
                ,direction='in'       # 设置刻度线方向，'in' 表示朝内
                ,width=2               # 设置刻度线粗细
                ,length=8              # 设置长短
                ,labelsize=20
                )
ax1.tick_params(which='minor'          # 设置对主刻度和次刻度生效
                ,bottom=True           # 底部坐标轴
                ,top=True
                ,left=True             # 左侧坐标轴
                ,right=True
                ,direction='in'       # 设置刻度线方向，'in' 表示朝内
                ,width=2               # 设置刻度线粗细
                ,length=5              # 设置长短
                ,labelsize=20
                )
ax1.xaxis.set_major_locator(ticker.MultipleLocator(2))
ax1.xaxis.set_minor_locator(ticker.MultipleLocator(1))
ax1.yaxis.set_major_locator(ticker.MultipleLocator(0.4))
ax1.yaxis.set_minor_locator(ticker.MultipleLocator(0.2))

ax1.set_xlabel("Distance (Å)")
ax1.set_ylabel("gr")
plt.plot(rdf_O.results.bins, rdf_O.results.rdf, alpha = 1,linewidth=2,color="green", linestyle = "-", label = "O-O")
plt.plot(rdf_H.results.bins, rdf_H.results.rdf, alpha = 1,linewidth=2,color="blue", linestyle = "-", label = "H-H")

plt.legend()

plt.savefig("./fig/1_gr_bulk.png")

out = pd.DataFrame.from_dict({"bins_O":rdf_O.results.bins, "rdf_O":rdf_O.results.rdf, "bins_H":rdf_H.results.bins, "rdf_H":rdf_H.results.rdf})

out.to_csv("./out/1_gr_bulk.csv",index=False)
